# Build and Test Servable

This notebook takes a model that was saved in tf1 and converts it to a tf2 model that is servable and encapsulated in a Docker image. Tests are also run to ensure that nothing was messed up in the conversion process.

## Development Environment

Create a new conda environment with the latest versions of `tensorflow`, `numpy`, `cv2`, and `matplotlib`.

In [1]:
# Test imports. 
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt

assert tf.__version__ == "2.0.0"

## Simple Check of the Model

Test out the model on a few images just to make sure it's creating reasonable predictions.

In [8]:
model = tf.saved_model.load("gender_model_2")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
model.signatures['serving_default'].inputs

[<tf.Tensor 'data:0' shape=(1, 224, 224, 3) dtype=float32>]

In [4]:
infer = model.signatures["serving_default"]
print(infer.structured_outputs)

{'output': <tf.Tensor 'prob:0' shape=(1, 2) dtype=float32>}


In [5]:
x = np.ones([1, 224, 224, 3]).astype('float32')
y = infer(tf.constant(x))

In [6]:
y['output'].numpy()[0]

array([0.31617698, 0.68382293], dtype=float32)

In [9]:
# Grab some pictures from the internet, turn them into tensors, and feed them to the model.
# The predictions seem reasonable: it looks like class 1 is female and class 2 is male.
for pic in ["man1.jpeg", "man2.jpeg", "man3.jpeg", "man4.jpeg", "carl.jpg",
            "woman1.jpg", "woman2.jpeg", "woman3.jpeg", "woman4.jpeg",
            "jiyoung.jpeg"]:

    img = cv2.imread(pic)
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3).astype('float32')
    y = infer(tf.constant(img))
    print(pic)
    print(y['output'].numpy()[0])

man1.jpeg
[0.00256076 0.99743927]
man2.jpeg
[0.00449356 0.9955064 ]
man3.jpeg
[0.00320393 0.9967961 ]
man4.jpeg
[0.00619051 0.9938095 ]
carl.jpg
[0.00196772 0.9980323 ]
woman1.jpg
[0.9972639  0.00273612]
woman2.jpeg
[0.22375257 0.77624744]
woman3.jpeg
[0.9957283  0.00427165]
woman4.jpeg
[0.9678138  0.03218627]
jiyoung.jpeg
[0.99224126 0.00775879]


In [16]:
# Save this as a new SavedModel
# https://www.tensorflow.org/guide/saved_model

# Make sure tp pass the serving signature!
tf.saved_model.save(model, "./gender_model_serving_2/1/", signatures=model.signatures)

INFO:tensorflow:Assets written to: ./gender_model_serving_2/1/assets


In [14]:
model_ = tf.saved_model.load("./gender_model_serving_2/1/")
model_.signatures

_SignatureMap({})

In [13]:
model.signatures

_SignatureMap({'serving_default': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x654cb6790>})

In [17]:
! saved_model_cli show --dir ./gender_model_serving_2/1/ --tag_set serve

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


## Convert to tf image

Add this model to a tensorflow docker image. This can then be easily deployed online. These commands follow [this documentation](https://www.tensorflow.org/tfx/serving/docker).

In [1]:
import json
import numpy
import requests
import cv2

url = "localhost"
port = "8080"
model = "gender_model"

# Mimic the shape of the incoming data. First axis are number of images.

for pic in ["cameron_screenshot.png", "man1.jpeg", "man2.jpeg", "man3.jpeg", "man4.jpeg", "carl.jpg",
            "woman1.jpg", "woman2.jpeg", "woman3.jpeg", "woman4.jpeg",
            "jiyoung.jpeg"]:

    img = cv2.imread("data/" + pic)
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3).astype('float32')
    print(pic)

    data = json.dumps({"signature_name": "serving_default",
                       "instances": img.tolist()})
    headers = {"content-type": "application/json"}
    json_response = requests.post(f"http://{url}:{port}/v1/models/{model}:predict", data=data, headers=headers)

    predictions = numpy.array(json.loads(json_response.text)["predictions"])

    print(predictions) # predictions are the same to a several decimal places.
    print(img.mean())

cameron_screenshot.png
[[0.02354481 0.97645515]]
69.68039
man1.jpeg
[[0.00256077 0.99743927]]
34.20193
man2.jpeg
[[0.00449357 0.99550641]]
129.97061
man3.jpeg
[[0.00320393 0.99679607]]
112.29698
man4.jpeg
[[0.00619052 0.99380952]]
219.36722
carl.jpg
[[0.00196772 0.99803227]]
124.49874
woman1.jpg
[[0.99726391 0.00273612]]
74.30365
woman2.jpeg
[[0.2237528 0.7762472]]
111.5687
woman3.jpeg
[[0.99572831 0.00427166]]
89.461266
woman4.jpeg
[[0.96781367 0.03218631]]
113.05363
jiyoung.jpeg
[[0.99224126 0.00775878]]
108.43575


## Run complete tests

Test the converted model against the original, generating the same statistics as in the original paper.

In [11]:
img = cv2.imread("data/" + pic)
img = cv2.resize(img, (224, 224))
# img = img.reshape(1, 224, 224, 3).astype('float32')



In [12]:
cv2.imshow('image', img)